In [2530]:
from ipyturtle import Turtle
import numpy as np
import random

random.seed(100)




In [2531]:
def setup(t):
    t.penup()
    t.back(20)
    move_left(t, 100)
    t.pendown()
    t.forward(200)
    t.back(200)
    move_right(t, 200)
    t.forward(200)
    t.back(200)
    move_left(t, 100)
    t.penup()
    t.forward(40)
    move_left(t, 100)

    for i in range(5):
    
        if(i % 2 == 0):
            move_right(t, 40)
            t.pendown()
            move_right(t, 120)
            t.penup()
            move_left(t, 160)
            t.forward(40)
    
        if(i % 2 == 1):
            t.pendown()
            move_right(t, 80)
            t.penup()
            move_right(t, 40)
            t.pendown()
            move_right(t, 80)
            t.penup()
            move_left(t, 200)
            t.forward(40)
            t.penup()

    t.back(220)
    move_right(t, 100)
    t.pendown()
    t.position()

def move_left(t, mult):
    t.left(90)
    t.forward(1 * mult)
    t.right(90)

def move_right(t, mult):
    t.right(90)
    t.forward(1 * mult)
    t.left(90)
    
def is_y(x, y):
    
    if(round(x) == round(y)):
        return(True)
    
    return(False)

def is_even(x):
    
    if(round(x) % 2 == 0):
        return(True)
    else:
        return(False)
    
def is_center_3(x, mult):
    if(is_y(x / mult, 0)):
        return True

    if(is_y(x / mult, -1)):
        return True

    if(is_y(x / mult, 1)):
        return True
    
    return False

def is_corner_left(x, mult):
    if(is_y(x / mult, -2)):
        return True
    
    return False

def is_corner_right(x, mult):
    if(is_y(x / mult, 2)):
        return True
    
    return False

def is_corner(x, mult):
    return(is_corner_right(x, mult) or is_corner_left(x, mult))

def is_center_1(x, mult):
    if(is_y(x / mult, 0)):
        return True

    return False

def is_center_2(x, mult):
    if(is_y(x / mult, -1)):
        return True

    if(is_y(x / mult, 1)):
        return True

    return False

def move(t, action):
    
    state = 0
    mult = 40
    points = 0
    
    if(action == 0):
        points = action_left(t, mult)
            
    if(action == 1):
        points = action_right(t, mult)
    
    if(action == 2):
        points = action_forward(t, mult)
        
    x = t.position()[0]
    y = t.position()[1]
    
   
    if((is_center_3(x, mult) and is_even(y / mult)) or (is_center_2(x, mult) and not is_even(y / mult))):
        state = 0
    
    if(is_corner_left(x, mult) and is_even(y / mult)):
        state = 1

    if(is_corner_right(x, mult) and is_even(y / mult)):
        state = 2

    if(is_corner_left(x, mult) and not is_even(y / mult)):
        state = 3

    if(is_corner_right(x, mult) and not is_even(y / mult)):
        state = 4
        
    if(is_center_1(x, mult) and not is_even(y / mult)):
        state = 5

    if(is_y(y, 5 * mult)):
        state = -1
       
    return((points, state))
    
def action_left(t, mult):
    
    x = t.position()[0] 
    y = t.position()[1]
    
    points = 0
    
    if(is_corner_left(x, mult)):
        points = -10
    else:
        points = 0
        move_left(t, mult)
        
    return(points)
    
def action_right(t, mult):
    
    x = t.position()[0] 
    y = t.position()[1]
    
    points = 0
    
    if(is_corner_right(x, mult)):
        points = -10
            
    else:
        points = 0
        move_right(t, mult)
        
    return(points)
   
    
def action_forward(t, mult):
    
    x = t.position()[0] 
    y = t.position()[1]
    
    points = 0
    
    if(is_y(y, 4 * mult) and (is_corner(x, mult))):
        t.forward(1 * mult)
        points = 100
        return(points)
          
    if(is_even(y / mult)):
        
        if(is_corner(x, mult)):
            t.forward(1 * mult)
            points = 10
        else:
            points = -10
    else:           
   
        if(is_y(x, 0)):
            t.forward(1 * mult)
            points = 10
        
        else:
            points = -10
                     
    
    
    
    return(points)


# Build the maze

In [2532]:
t = Turtle(width = 400, height = 400)
t

Turtle()

In [2533]:
setup(t)

# One walk through the Maze

In [2534]:
for i in range(5):
    move(t, i % 2)
    move(t, i % 2)
    move(t, 2)


# Random walk through the maze

In [2535]:
runs = 100
steps_needed_random = np.zeros(100)

for i in range(runs):
    t.reset()
    setup(t)
    end = False
    cnt = 0
    while(not end):
        action = random.randint(0, 2)
        points, state = move(t, action)
        if(state == -1):
            end = True
        cnt= cnt + 1
    
    steps_needed_random[i] = cnt

In [2537]:
print(np.mean(steps_needed_random))

95.41


# Simple learning algorithm

In [2541]:
def simple_learning(t, q_table, state = 0, steps = 1, verbose = False, \
                    alpha = 1, gamma = 0.1, epsilon = 0.2, adapt = True):
    
    
    i = 0
    
    end = False
    
    while((i < steps) and (not end)):
        
       
        if(state == -1):
            print("Game ended")
            end = True
        
        row = state
                
        rand_numb = random.random()
        
        if(rand_numb < epsilon):
            action = random.randint(0, q_table.shape[0]-1)
        else:
            rows, = np.where(q_table[row, :] == np.max(q_table[row, : ]))
            ind = random.randint(0, rows.shape[0]-1)
            action = rows[ind]
        
        if(action > 2):
            action = 2
        
        points, state = move(t, action)
        
        if(verbose):
            print("action:" + str(action))
            print("points:" + str(points))
            print("state:" + str(state))
            print("row:" + str(row))
        
        if(adapt):
            q_table[row, action] = (1 - alpha) * q_table[row, action] + \
                               alpha * (points + gamma * np.max(q_table[state,:])) 
    
        i = i + 1
        
    
    if(verbose):
        print("Necessary steps: "+str(i))
    
    return(q_table, state)
        
       

# Setup learning

In [2481]:
q_table = np.zeros((6, 3))
state = 0

# Learn

In [2516]:
t.reset()
setup(t)
state = 0
cnt = 0

In [2517]:
#cnt = cnt + 1
#print("step: " + str(cnt))
q_table, state = simple_learning(t, q_table, state, 400, False, alpha = 0.2, epsilon = 0.5)
print(np.round(q_table))

Game ended
Necessary steps: 37
[[  1.   0. -10.]
 [ -6.   0.  39.]
 [  0.  -4.  42.]
 [-10.   0. -10.]
 [  0. -10. -10.]
 [ -0.  -1.   2.]]


# Does it work better now?

In [2539]:
runs = 100
steps_needed_learning = np.zeros(100)

for i in range(runs):
    state = 0
    t.reset()
    setup(t)
    end = False
    cnt = 0
    while(not end):
        q_table, state = simple_learning(t, q_table, state, 1, False, alpha = 0.2, epsilon = 0.5)
        if(state == -1):
            end = True
        cnt= cnt + 1
    
    steps_needed_learning[i] = cnt



Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary st

Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary st

Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary st

Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary steps: 1
Necessary st

In [2543]:
print(np.mean(steps_needed_learning))
print(np.mean(steps_needed_random))

58.89
95.41
